In [1]:
# load basic libraries 
import pandas as pd
import numpy as np

# For feature slection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# For hyperparemeter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score

# For evaluating model performance
from sklearn.metrics import make_scorer, r2_score

# load modelling libraries
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor
import xgboost

# To visualise all the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
# load train data set
train=pd.read_csv('X_train_clean_data.csv')

# Load test data set
test=pd.read_csv('X_test_clean_data.csv')

In [3]:
train.head()

,SalePrice,LotFrontage,LotArea,Street,Alley,LotShape,Utilities,LotConfig,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,Total_SF,TotalFloorSF,TotalPorchSF,TotalBathrooms,HasBasement,HasGarage,HasFireplace,HasPorch,HasPool,WasRemodeled,WasCompleted,HasPavedDrive,MSSubClass,MSZoning,LandContour,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,12.247699,0.518594,0.468234,1.0,0.0,1.000000,1.0,0.00,1.0,0.666667,0.500,0.364180,0.435859,0.716038,0.666667,0.5,0.8,0.75,0.25,1.000000,0.853057,0.166667,0.0,0.646838,0.857442,1.00,1.0,1.0,0.435269,0.899422,0.0,0.684506,0.5,0.00000,0.792481,0.63093,0.630930,0.5,0.666667,0.682606,1.0,0.0,0.0,0.833333,0.382680,0.666667,0.682606,0.871552,0.6,0.6,1.0,0.000000,0.699526,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.724978,0.684506,0.634088,0.647313,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.000000,0.84138,0.477757,0.599902,0.604836,1.0,0.949399,1.000000,0.0,0.0,0.995560,1.000000,0.508175,1.000000,1.0,1.0,0.685280,0.209046,0.245811,0.321751
1,12.109016,0.615266,0.500160,1.0,0.0,1.000000,1.0,0.75,1.0,0.555556,0.875,0.704421,0.843065,0.000000,0.333333,0.5,0.8,0.75,1.00,0.833333,0.895378,0.166667,0.0,0.728731,0.906680,1.00,1.0,1.0,0.614977,0.000000,0.0,0.557071,0.0,0.63093,0.792481,0.00000,0.630930,0.5,0.333333,0.526456,1.0,0.5,0.6,0.833333,0.740205,0.666667,0.682606,0.847414,0.6,0.6,1.0,0.843935,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.719104,0.557071,0.000000,0.446705,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.673540,0.84138,0.477757,0.545994,0.181268,1.0,0.949399,0.519097,0.0,0.0,0.491781,0.458722,0.000000,0.462004,1.0,1.0,0.275509,1.000000,0.245811,0.321751
2,12.317171,0.539577,0.539845,1.0,0.0,0.666667,1.0,0.00,1.0,0.666667,0.500,0.422652,0.473357,0.690361,0.666667,0.5,0.8,0.75,0.50,1.000000,0.804590,0.166667,0.0,0.783246,0.866587,1.00,1.0,1.0,0.468644,0.901279,0.0,0.702749,0.5,0.00000,0.792481,0.63093,0.630930,0.5,0.666667,0.526456,1.0,0.5,0.6,0.833333,0.444123,0.666667,0.682606,0.885882,0.6,0.6,1.0,0.000000,0.637503,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.743883,0.702749,0.577867,0.647313,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.84138,0.477757,0.599902,0.604836,1.0,0.949399,1.000000,0.0,0.0,0.995560,1.000000,0.508175,1.000000,1.0,1.0,1.000000,0.209046,0.245811,0.321751
3,11.849405,0.481406,0.498853,1.0,0.0,0.666667,1.0,0.25,1.0,0.666667,0.500,0.919066,0.878382,0.000000,0.333333,0.5,0.6,1.00,0.25,0.833333,0.699487,0.166667,0.0,0.811361,0.841689,0.75,1.0,1.0,0.488827,0.883204,0.0,0.686220,0.5,0.00000,0.500000,0.00000,0.630930,0.5,0.666667,0.609423,1.0,0.5,0.8,0.166667,0.469279,0.333333,0.861353,0.893388,0.6,0.6,1.0,0.000000,0.607387,0.888227,0.0,0.0,0.0,0.0,0.0,0.0,0.711840,0.686220,0.880366,0.323657,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.582479,0.84138,0.477757,0.637338,0.604836,1.0,0.949399,1.000000,0.0,0.0,0.441421,0.505772,0.000000,0.246106,1.0,1.0,0.685280,0.993235,0.245811,0.000000
4,12.429220,0.638020,0.599170,1.0,0.0,0.666667,1.0,0.75,1.0,0.777778,0.500,0.446592,0.534491,0.794318,0.666667,0.5,0.8,0.75,0.75,1.000000,0.843322,0.166667,0.0,0.798858,0.894337,1.00,1.0,1.0,0.569929,0.927299,0.0,0.789834,0.5,0.00000,0.792481,0.63093,0.732487,0.5,0.666667,0.748070,1.0,0.5,0.6,0.833333,0.469279,0.666667,0.861353,0.929819,0.6,0.6,1.0,0.779126,0.753005,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.819119,0.789834,0.682564,0.647313,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.000000,0.84138,0.477757,1.000000,0.60

In [4]:
test.head()

,Id,LotFrontage,LotArea,Street,Alley,LotShape,Utilities,LotConfig,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,Total_SF,TotalFloorSF,TotalPorchSF,TotalBathrooms,HasBasement,HasGarage,HasFireplace,HasPorch,HasPool,WasRemodeled,WasCompleted,HasPavedDrive,MSSubClass,MSZoning,LandContour,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,1461,0.615266,0.547985,1.0,0.0,1.000000,1.0,0.00,1.0,0.444444,0.625,0.795130,0.951628,0.000000,0.333333,0.5,0.6,0.75,0.25,0.500000,0.799694,0.333333,0.682079,0.722237,0.861237,0.50,1.0,1.0,0.456409,0.000000,0.0,0.413434,0.0,0.0,0.500000,0.00000,0.50000,0.5,0.333333,0.430677,1.0,0.0,0.0,0.833333,0.835522,0.333333,0.430677,0.911110,0.6,0.6,1.0,0.732650,0.000000,0.0,0.0,0.787610,0.0,0.0,0.75,0.000000,0.594431,0.413434,0.736820,0.000000,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.67354,0.00000,0.477757,0.340605,0.181268,1.0,0.949399,0.519097,0.0,0.0,0.995560,1.000000,0.000000,0.462004,1.0,1.000000,0.461761,0.0,0.245811,0.321751
1,1462,0.621058,0.599293,1.0,0.0,0.666667,1.0,0.25,1.0,0.555556,0.625,0.806973,0.965802,0.635823,0.333333,0.5,0.6,0.75,0.25,0.833333,0.887860,0.166667,0.000000,0.774669,0.913242,0.50,1.0,1.0,0.638929,0.000000,0.0,0.578768,0.0,0.0,0.500000,0.63093,0.63093,0.5,0.666667,0.526456,1.0,0.0,0.0,0.833333,0.847967,0.333333,0.430677,0.793918,0.6,0.6,1.0,0.884782,0.612031,0.0,0.0,0.000000,0.0,0.0,0.00,0.977707,0.737514,0.578768,0.554777,0.178121,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.67354,0.84138,0.477757,0.340605,0.604836,1.0,0.949399,0.519097,1.0,0.0,0.441421,0.416062,0.508175,0.462004,1.0,0.072672,0.461761,0.0,0.245811,0.321751
2,1463,0.578937,0.591508,1.0,0.0,0.666667,1.0,0.00,1.0,0.444444,0.500,0.536396,0.623943,0.000000,0.333333,0.5,0.8,0.75,0.25,1.000000,0.867818,0.166667,0.000000,0.635232,0.867685,0.75,1.0,1.0,0.472652,0.873154,0.0,0.664148,0.0,0.0,0.792481,0.63093,0.63093,0.5,0.333333,0.526456,1.0,0.5,0.6,0.833333,0.563644,1.000000,0.682606,0.853855,0.6,0.6,1.0,0.793724,0.602612,0.0,0.0,0.000000,0.0,0.0,0.75,0.000000,0.723727,0.664148,0.546240,0.446705,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.00000,0.84138,0.477757,0.593081,0.604836,1.0,0.949399,1.000000,0.0,0.0,0.995560,1.000000,0.000000,1.000000,1.0,1.000000,0.581915,0.0,0.245811,0.321751
3,1464,0.603464,0.509823,1.0,0.0,0.666667,1.0,0.00,1.0,0.555556,0.625,0.521333,0.623943,0.412627,0.333333,0.5,0.6,0.75,0.25,1.000000,0.832369,0.166667,0.000000,0.745663,0.867411,1.00,1.0,1.0,0.471653,0.868716,0.0,0.657660,0.0,0.0,0.792481,0.63093,0.63093,0.5,0.666667,0.609423,1.0,0.5,0.8,0.833333,0.547817,1.000000,0.682606,0.850379,0.6,0.6,1.0,0.871832,0.612031,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.719949,0.657660,0.554777,0.446705,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.00000,0.84138,0.477757,0.593081,0.604836,1.0,0.949399,1.000000,0.0,0.0,0.995560,1.000000,0.508175,1.000000,1.0,1.000000,0.461761,0.0,0.245811,0.321751
4,1465,0.327196,0.337197,1.0,0.0,0.666667,1.0,0.00,1.0,0.777778,0.500,0.598466,0.716256,0.000000,0.666667,0.5,0.8,0.75,0.25,0.833333,0.724978,0.166667,0.000000,0.892862,0.908477,1.00,1.0,1.0,0.621535,0.000000,0.0,0.563011,0.0,0.0,0.792481,0.00000,0.50000,0.5,0.666667,0.430677,1.0,0.0,0.0,0.833333,0.628867,0.666667,0.682606,0.860556,0.6,0.6,1.0,0.000000,0.748969,0.0,0.0,0.817326,0.0,0.0,0.00,0.000000,0.724144,0.563011,0.833483,0.323657,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.81676,0.84138,1.000000,0.967013,0.604836,1.

In [5]:
# drop target feature from train data set
X_train = train.drop(['SalePrice'],axis=1)

# Capture the target feature
y_train = train['SalePrice']

In [6]:
# Shape of X_train, y_train
print("X_train have {} rows and {} columns".format(X_train.shape[0],X_train.shape[1]))
print("y_train have {} rows and {} column".format(y_train.shape[0],1))

X_train have 1444 rows and 91 columns
y_train have 1444 rows and 1 column


# Creating Some Function

In [7]:
# Creating a function to calculate root mean square error
def rmse(model, x=X_train, y=y_train):
  cv_scores = -cross_val_score(model, x, y, scoring='neg_mean_squared_error', cv=10)
  return np.round(np.sqrt(cv_scores),6)


# Create a function to evaluate model performance
def train_model(model, X_train=X_train, y_train=y_train):
    cv = KFold(n_splits = 3, shuffle=True, random_state = 45)
    r2 = make_scorer(r2_score)
    r2_val_score = cross_val_score(model, X_train, y_train, cv=cv, scoring = r2)
    score = [r2_val_score.mean()]
    return score

# Feature Selection using Lasso

In [8]:
param_grid = {'alpha':[0.0001,0.001,0.01,1.,5.,10.,25.,0.005,0.00055],'max_iter':[50000]}
lasso = GridSearchCV(Lasso(), cv=5, param_grid=param_grid, scoring='neg_mean_squared_error')
lasso.fit(X_train, y_train)
alpha = lasso.best_params_['alpha']

# Home in
param_grid = {'alpha':[x/100. * alpha for x in range(50,150,5)],'max_iter':[50000]}
lasso = GridSearchCV(Lasso(), cv=5, param_grid=param_grid, scoring='neg_mean_squared_error')
lasso.fit(X_train, y_train)
alpha = lasso.best_params_['alpha']
lasso = lasso.best_estimator_

print('Lasso -> Train RMSE: {:,.5f}| alpha {:,.5f}'.format(rmse(lasso).mean(),alpha))


Lasso -> Train RMSE: 0.11238| alpha 0.00015


In [9]:
# Apply Feature Selection
# first, specify the Lasso Regression model, and select a suitable alpha (penalty).
# The bigger the alpha the less features that will be selected.

# Then I use the selectFromModel object from sklearn, which
# will select the features which coefficients are non-zero

feature_sel_model = SelectFromModel(Lasso(alpha=0.00015, random_state=0))
feature_sel_model.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.00015, copy_X=True, fit_intercept=True,
                                max_iter=1000, normalize=False, positive=False,
                                precompute=False, random_state=0,
                                selection='cyclic', tol=0.0001,
                                warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [10]:
# let's print the number of total, selected and rejected features

selected_feat = X_train.columns[(feature_sel_model.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(np.sum(feature_sel_model.estimator_.coef_ == 0)))

total features: 91
selected features: 71
features with coefficients shrank to zero: 20


In [11]:
# Selected features
selected_feat

Index(['LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LotConfig',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'HeatingQC', 'CentralAir',
       'Electrical', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'MiscVal', 'Total_SF', 'TotalFloorSF', 'TotalPorchSF', 'HasBasement',
       'HasGarage', 'HasFireplace', 'HasPorch', 'WasRemodeled', 'WasCompleted',
       'MSZoning', 'LandContour', 'Neighborhood', 'Condition1', 'BldgType',
       'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Founda

In [12]:
X_train=X_train[selected_feat]
X_train.head()

,LotFrontage,LotArea,Street,Alley,LotShape,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,HeatingQC,CentralAir,Electrical,1stFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,ScreenPorch,PoolArea,PoolQC,MiscVal,Total_SF,TotalFloorSF,TotalPorchSF,HasBasement,HasGarage,HasFireplace,HasPorch,WasRemodeled,WasCompleted,MSZoning,LandContour,Neighborhood,Condition1,BldgType,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,MiscFeature,MoSold,YrSold,SaleCondition
0,0.518594,0.468234,1.0,0.0,1.000000,0.00,0.666667,0.500,0.364180,0.435859,0.716038,0.666667,0.5,0.8,0.25,1.000000,0.853057,0.0,0.646838,1.00,1.0,1.0,0.435269,0.0,0.684506,0.5,0.792481,0.63093,0.630930,0.5,0.666667,1.0,0.0,0.0,0.833333,0.382680,0.666667,0.682606,0.6,0.6,1.0,0.000000,0.699526,0.0,0.0,0.0,0.0,0.724978,0.684506,0.634088,1.0,1.0,0.0,1.0,0.0,0.0,0.84138,0.477757,0.599902,0.604836,0.949399,0.0,0.995560,1.000000,0.508175,1.000000,1.0,1.0,0.685280,0.209046,0.321751
1,0.615266,0.500160,1.0,0.0,1.000000,0.75,0.555556,0.875,0.704421,0.843065,0.000000,0.333333,0.5,0.8,1.00,0.833333,0.895378,0.0,0.728731,1.00,1.0,1.0,0.614977,0.0,0.557071,0.0,0.792481,0.00000,0.630930,0.5,0.333333,1.0,0.5,0.6,0.833333,0.740205,0.666667,0.682606,0.6,0.6,1.0,0.843935,0.000000,0.0,0.0,0.0,0.0,0.719104,0.557071,0.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.84138,0.477757,0.545994,0.181268,0.949399,0.0,0.491781,0.458722,0.000000,0.462004,1.0,1.0,0.275509,1.000000,0.321751
2,0.539577,0.539845,1.0,0.0,0.666667,0.00,0.666667,0.500,0.422652,0.473357,0.690361,0.666667,0.5,0.8,0.50,1.000000,0.804590,0.0,0.783246,1.00,1.0,1.0,0.468644,0.0,0.702749,0.5,0.792481,0.63093,0.630930,0.5,0.666667,1.0,0.5,0.6,0.833333,0.444123,0.666667,0.682606,0.6,0.6,1.0,0.000000,0.637503,0.0,0.0,0.0,0.0,0.743883,0.702749,0.577867,1.0,1.0,1.0,1.0,1.0,0.0,0.84138,0.477757,0.599902,0.604836,0.949399,0.0,0.995560,1.000000,0.508175,1.000000,1.0,1.0,1.000000,0.209046,0.321751
3,0.481406,0.498853,1.0,0.0,0.666667,0.25,0.666667,0.500,0.919066,0.878382,0.000000,0.333333,0.5,0.6,0.25,0.833333,0.699487,0.0,0.811361,0.75,1.0,1.0,0.488827,0.0,0.686220,0.5,0.500000,0.00000,0.630930,0.5,0.666667,1.0,0.5,0.8,0.166667,0.469279,0.333333,0.861353,0.6,0.6,1.0,0.000000,0.607387,0.0,0.0,0.0,0.0,0.711840,0.686220,0.880366,1.0,1.0,1.0,1.0,1.0,0.0,0.84138,0.477757,0.637338,0.604836,0.949399,0.0,0.441421,0.505772,0.000000,0.246106,1.0,1.0,0.685280,0.993235,0.000000
4,0.638020,0.599170,1.0,0.0,0.666667,0.75,0.777778,0.500,0.446592,0.534491,0.794318,0.666667,0.5,0.8,0.75,1.000000,0.843322,0.0,0.798858,1.00,1.0,1.0,0.569929,0.0,0.789834,0.5,0.792481,0.63093,0.732487,0.5,0.666667,1.0,0.5,0.6,0.833333,0.469279,0.666667,0.861353,0.6,0.6,1.0,0.779126,0.753005,0.0,0.0,0.0,0.0,0.819119,0.789834,0.682564,1.0,1.0,1.0,1.0,0.0,0.0,0.84138,0.477757,1.000000,0.604836,0.949399,0.0,0.995560,1.000000,0.508175,1.000000,1.0,1.0,0.778204,0.209046,0.321751


In [13]:
# Drop unnecessary columns from test data set
X_features= [feature for feature in test.columns if feature not in selected_feat]
X_test=test.drop(X_features,axis=1)

print("X_test have {} rows and {} columns".format(X_test.shape[0],X_test.shape[1]))

X_test have 1459 rows and 71 columns


# Model Building

# Linear Regression

In [14]:
# Linear Regression
LR = linear_model.LinearRegression()
LR_score = train_model(LR)

LR_rsme = rmse(LR)

print('Score: {:.5f}'.format((LR_score[0])))
print('RMSE: {:.5f}'.format(LR_rsme.mean()))

Score: 0.91070
RMSE: 42823620173.71836


Ridge Regression

In [15]:
ridge_LR = linear_model.Ridge()
ridge_LR_score=train_model(ridge_LR)
ridge_LR_rmse=rmse(ridge_LR)

print('Score: {:.5f}'.format((ridge_LR_score[0])))
print('RMSE: {:.5f}'.format(ridge_LR_rmse.mean()))

Score: 0.91176
RMSE: 0.11311


Lasso Regression

In [16]:
lasso_LR = linear_model.Lasso(alpha=0.00015)
lasso_LR_score=train_model(lasso_LR)
lasso_LR_rmse= rmse(lasso_LR)

print('Score: {:.5f}'.format((lasso_LR_score[0])))
print('RMSE: {:.5f}'.format(lasso_LR_rmse.mean()))

Score: 0.91325
RMSE: 0.11239


# Support Vector Regressor

In [17]:
svr_reg = SVR(kernel='rbf')
svr_reg_score=train_model(svr_reg)
svr_reg_rmse=rmse(svr_reg)

print('Score: {:.5f}'.format((svr_reg_score[0])))
print('RMSE: {:.5f}'.format(svr_reg_rmse.mean()))

Score: 0.90126
RMSE: 0.11780


SVR Hyperparameter Tuning 

In [18]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
params = {'kernel': ['linear', 'poly', 'rbf'],
         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
         'C': [0.1, 1, 10, 100, 1000],
         'epsilon': [1, 0.2, 0.1, 0.01, 0.001, 0.0001]}
rand_search = RandomizedSearchCV(SVR(), param_distributions=params, n_jobs=-1, cv=10)
rand_search.fit(X_train, y_train)
rand_search.best_score_

0.9162450738653998

In [19]:
rand_search.best_estimator_

SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0001,
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [20]:
svr_reg_tuning1=SVR(C=0.1, cache_size=200, coef0=0.0, degree=3, epsilon=0.0001, gamma=0.1,
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

svr_reg_tuning_score=train_model(svr_reg_tuning1)
svr_reg_tuning_rmse= rmse(svr_reg_tuning1)

print('Score: {:.5f}'.format((svr_reg_tuning_score[0])))
print('RMSE: {:.5f}'.format(svr_reg_tuning_rmse.mean()))

Score: 0.90908
RMSE: 0.11340


Observation: After tuning the SVR R2 score is imporved and rmse is reduced which is good.

# Decision Tree Regressor

In [21]:
dt_reg = DecisionTreeRegressor(random_state=21)
dt_reg_score=train_model(dt_reg)
dt_reg_rmse= rmse(dt_reg)

print('Score: {:.5f}'.format((dt_reg_score[0])))
print('RMSE: {:.5f}'.format(dt_reg_rmse.mean()))

Score: 0.73645
RMSE: 0.19227


# Random Forest Regressor

In [22]:
rf_reg = RandomForestRegressor(n_estimators = 1000, random_state=51)
rf_reg_score=train_model(rf_reg)
rf_reg_rmse= rmse(rf_reg)

print('Score: {:.5f}'.format((rf_reg_score[0])))
print('RMSE: {:.5f}'.format(rf_reg_rmse.mean()))

Score: 0.87888
RMSE: 0.13108


# Bagging & boosting

In [23]:
# Bagging
br_reg = BaggingRegressor(n_estimators=1000, random_state=51)
br_reg_score=train_model(br_reg)
br_reg_rmse= rmse(br_reg)

print('Score: {:.5f}'.format((br_reg_score[0])))
print('RMSE: {:.5f}'.format(br_reg_rmse.mean()))

print('\n')

# Boosting
gbr_reg = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, loss='ls', random_state=51)
gbr_reg_score=train_model(gbr_reg)
gbr_reg_rmse= rmse(gbr_reg)

print('Score: {:.5f}'.format((gbr_reg_score[0])))
print('RMSE: {:.5f}'.format(gbr_reg_rmse.mean()))

Score: 0.87886
RMSE: 0.13120


Score: 0.89794
RMSE: 0.11863


# XGBoost

In [24]:
xgb_reg = xgboost.XGBRegressor(bbooster='gbtree', random_state=51)
xgb_reg_score=train_model(xgb_reg)
xgb_reg_rmse= rmse(xgb_reg)

print('Score: {:.5f}'.format((xgb_reg_score[0])))
print('RMSE: {:.5f}'.format(xgb_reg_rmse.mean()))

[00:00:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bbooster } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:00:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bbooster } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:00:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { bbooster } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

XGBoost Hyper perameter tuning

In [25]:
xgb2_reg=xgboost.XGBRegressor(n_estimators= 899,
 mon_child_weight= 2,
 max_depth= 4,
 learning_rate= 0.05,
 booster= 'gbtree')


xgb2_reg_score=train_model(xgb2_reg)
xgb2_reg_rmse= rmse(xgb2_reg)

print('Score: {:.5f}'.format((xgb2_reg_score[0])))
print('RMSE: {:.5f}'.format(xgb2_reg_rmse.mean()))

[00:00:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { mon_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:00:41] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { mon_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:00:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { mon_child_weight } might not be used.

  This may not be accurate due to some parameters are onl

In [26]:
xgb3_reg = xgboost.XGBRegressor(colsample_bytree=0.2, 
                        gamma=0.0,
                        learning_rate=0.05,
                        max_depth=6,
                        min_child_weight=1.5,
                        n_estimators=7200,
                        reg_alpha=0.9,
                        reg_lambda=0.6,
                        subsample=0.2,
                        seed=42,
                        silent=1)
xgb3_reg_score=train_model(xgb3_reg)
xgb3_reg_rmse= rmse(xgb3_reg)

print('Score: {:.5f}'.format((xgb3_reg_score[0])))
print('RMSE: {:.5f}'.format(xgb3_reg_rmse.mean()))

[00:01:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:01:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[00:01:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

In [27]:
results = pd.DataFrame({
    'Model': ['XGBoost', 'Gradient Boosting Regressor ', 'Bagging Regressor','Random Forest','Decision Tree', 
              'Support Vector Regressor','Support Vector Tuned Regressor','Ridge LR','Lasso LR','Linear Regression',
              'XGBoost Tuned 2','XGBoost Tuned 3'],
              
    'Score': [xgb_reg_score[0], gbr_reg_score[0], br_reg_score[0], rf_reg_score[0], dt_reg_score[0], svr_reg_score[0],
              svr_reg_tuning_score[0], ridge_LR_score[0], lasso_LR_score[0], LR_score[0], xgb2_reg_score[0],xgb3_reg_score[0]],
    
    'RSME': [xgb_reg_rmse.mean(), gbr_reg_rmse.mean(), br_reg_rmse.mean(), rf_reg_rmse.mean(), dt_reg_rmse.mean(),
             svr_reg_rmse.mean(), svr_reg_tuning_rmse.mean(), ridge_LR_rmse.mean(), lasso_LR_rmse.mean(), LR_rsme.mean(),
             xgb2_reg_rmse.mean(), xgb3_reg_rmse.mean()]
            })

result = results.sort_values(by='RSME', ascending=True)
result = result.set_index('Model')
display(result)

,Score,RSME
Model,,
Lasso LR,0.913254,1.123876e-01
Ridge LR,0.911756,1.131132e-01
Support Vector Tuned Regressor,0.909079,1.133998e-01
XGBoost Tuned 3,0.905316,1.144155e-01
XGBoost Tuned 2,0.906135,1.149445e-01
Support Vector Regressor,0.901256,1.178028e-01
Gradient Boosting Regressor,0.897939,1.186312e-01
XGBoost,0.879787,1.287967e-01
Random Forest,0.878885,1.310762e-01


Observation: Lasso LR is giving least rmse, so we go with it for predicting sale price of test data set

In [28]:
# Fit lasso_LR on train data set
lasso_LR.fit(X_train,y_train)

# predict sale price of test data set
y_pred = np.exp(lasso_LR.predict(X_test)).round(2)

# Save results in csv format
submit_test = pd.concat([test['Id'],pd.DataFrame(y_pred)], axis=1)
submit_test.columns=['Id', 'SalePrice']
submit_test.to_csv('sample_submission.csv', index=False)
submit_test.head()

,Id,SalePrice
0,1461,101943.80
1,1462,160693.13
2,1463,178976.33
3,1464,197393.07
4,1465,201834.15
